# **この Notebook は、[Google Colab: PDF to CSV 変換器を Colab に設置 \[第四話 望郷篇\] – Pandas を SQL っぽく使う](https://ggcs.io/2020/08/26/google-colab-pdf-export-04/) の手順詳細版です。**

- Website: ごたごた気流調査所  https://ggcs.io
- GitHub : Gota Gota Current Survey  https://github.com/ggcurrs/gota2-observatory
- Version 1.0.0
- Date   : 2020-08-26
- Updated: None

# 今回（最終回）の作業の流れ
- 前回（[乱麻篇](https://ggcs.io/2020/08/19/google-colab-pdf-export-03/)）までの作業で、tabula を使ってオリジナルの PDF をデータ化し、州名略記（state_code）> 州ごとの連番（sc_num）の順にソートするところまで漕ぎ着けました（df_sorted_intermed.csv として一時保存）。
- 最終回となる今回は、業種列（biz_type column）の整理と正規化を行います。


## 前提
- Google Colab 上で作業する前提です。
- [tabula-py](https://pypi.org/project/tabula-py/) がインストールされていること（*cf.* [立志篇](https://ggcs.io/2020/08/05/google-colab-pdf-export-01/)）。
- df_sorted_intermed.csv が Google Drive の　/content/drive/My Drive/pdf_project/data/df_sorted_intermed.csv に置いてあること（*cf.* [乱麻篇](https://ggcs.io/2020/08/19/google-colab-pdf-export-03/)）。


## 手順
0. 準備（Script 0）
1. 作業中のデータの読み込み（Script 1）
2. 業種列 (biz_type column) の Cleansing
  - 状況確認（Script 2）
  - Numeric系の整理（Scripts 3-4）
  - Non-numeric系の整理（Script 5-9）
3. 親会社列 (com_jp column) の Cleansing（Script 10）
4. Master table の作成
  - Master table 用 DF の準備（Script 11）
  - Non_numeric系修正結果反映（Script 12）
  - biz_type columnの正規化（Scripts 13-15）
5. ファイル保存（Scripts 16-17）

# Setup

In [ ]:
# Script 0
'''以下はすべて Google Colab / Drive が前提となっているので、
local machine で実験する時には適当に path などを変えてください。'''

# Google Drive のマウント。
from google.colab import drive
drive.mount('/content/drive')

# Modules の import.
import os
import pandas as pd
# ディレクトリ構造を定義する。
PROJECT_ROOT_PATH = '/content/drive/My Drive/pdf_project'
DATA_PATH   = os.path.join(PROJECT_ROOT_PATH, 'data')

print('準備完了 🍻')

## 作業中のデータの読み込み

- 前回保存した[作業ファイル](https://ggcs.io/2020/08/19/google-colab-pdf-export-03/#work_file)（df_sorted_intermed.csv）を読み込みます（Script 1)。
- せっかく sort してあるので、index をコピーして id column を作っておきます。

In [ ]:
# Script 1

df = pd.read_csv(os.path.join(DATA_PATH, 'df_sorted_intermed.csv'))
# Create a 'key' column
df.reset_index(drop=False, inplace=True)
df.rename(columns={'index': 'id'}, inplace=True)
df.head()

# biz_type column の Cleansing

## 状況確認
まずは biz_type column の状況を確認（Script 2）すると、
  - 数字で始まっているもの（例: '01  農業、林業、...）
  - 数字で始まっていないもの（例: '/  Wholesale...'）

の 2 種類があり、数字で始まっているタイプのもの（以下「***Numeric系***」）が元々のオリジナル（又はそれに近いもの）で、それ以外のもの（以下「***Non-numeric系***」）は [Triad退治](https://ggcs.io/2020/08/11/google-colab-pdf-export-02#scr004) などの際に大きく乱れてしまったものだと推測されます。

In [ ]:
# Script 2

list_biz_type = sorted(list(df.biz_type.unique()))
list_biz_type

['/  Wholesale (trading company), 11 非鉄金属製造業/  Manufacturing -  Non-metal products, 23 卸売業(商社)',
 '/  Wholesale (trading company), 22 運輸業/  Transportation, 18 輸送機械器具製造業/  Manufacturing – automobile , two',
 '/  Wholesale (trading company), 26 金融業、保険業/  Finance and insurance, 17 情報通信機械器具、電子部品・デバイス・電子回路製造業/',
 '0',
 '01  農業、林業、漁業/  Agriculture, forestry, fishery',
 '03 建設業/  Construction industry',
 '04 食料品、飲料・たばこ・飼料製造業/  Manufacturing – Food,, drink, tobacco',
 '04 食料品、飲料・たばこ・飼料製造業/  Manufacturing – Food,<br>drink, tobacco',
 '05 繊維工業/  Textile industry',
 '06 木材・木製品、パルプ・紙・紙加工品製造業/  Manufacturing-, paper and paper products',
 '06 木材・木製品、パルプ・紙・紙加工品製造業/  Manufacturing-<br>paper and paper products',
 '07 化学工業/  Chemical industry',
 '08 石油製品・石炭製品製造業/  Manufacturing – coal products,, petro products',
 '08 石油製品・石炭製品製造業/  Manufacturing – coal products,<br>petro products',
 '09 窯業・土石製品製造業/  Pottery',
 '10 鉄鋼業/  Steel',
 '11 非鉄金属製造業/  Manufacturing -  Non-metal products',
 '12 金属製品製造業/  Manufact

## Numeric系の整理
- そこで、まず数字で始まっているものを集めて CSV に落とし（Script 3）、オリジナル PDF と見比べながら、ここは手作業で修正して biz_type カテゴリー項目のマスターを作ります（完成したものが Script 4）。
- こちらは特に問題なさそうなので、この作業はここでいったん置いておきます。


In [ ]:
#Script 3

# Numeric系の list 化。
# list_biz_type から iセルを取り出し、最初の文字（i[0]）が数字だったら
# そいつは Numeric系だから、list に加える。
list_btyp_num = [i for i in list_biz_type if i[0].isnumeric()]

# CSV に落として local PC 上で手作業で整理 😩
pd.DataFrame(list_btyp_num).to_csv(
    os.path.join(DATA_PATH, 'list_btyp_num.csv'), index=False)

print('DONE!')

In [ ]:
#Script 4

# 手作業で整理した Numeric系データを DF にしたもの
table_biz_types = pd.DataFrame(
  [
    ['00', '(blank)'],
    ['01', '農業、林業、漁業 / Agriculture, forestry, fishery'],
    ['02', '(blank)'],
    ['03', '建設業 / Construction industry'],
    ['04', '食料品、飲料・たばこ・飼料製造業 / Manufacturing - Food, drink, tobacco'],
    ['05', '繊維工業 / Textile industry'],
    ['06', '木材・木製品、パルプ・紙・紙加工品製造業 / Manufacturing - paper and paper products'],
    ['07', '化学工業 / Chemical industry'],
    ['08', '石油製品・石炭製品製造業 / Manufacturing - coal products, petro products'],
    ['09', '窯業・土石製品製造業 / Pottery'],
    ['10', '鉄鋼業 / Steel'],
    ['11', '非鉄金属製造業 / Manufacturing - Non-metal products'],
    ['12', '金属製品製造業 / Manufacturing metal products'],
    ['13', 'はん用機械器具製造業 / Manufacturing - general machinery'],
    ['14', '生産用機械器具製造業 / Manufacturing - production machinery'],
    ['15', '業務用機械器具製造業 / Manufacturing - commercial machinery'],
    ['16', '電気機械器具製造業 / Manufacturing - electronic machinery'],
    ['17', '情報通信機械器具、電子部品・デバイス・電子回路製造業 / Manufacturing - IT products and electronic products'],
    ['18', '輸送機械器具製造業 / Manufacturing - automobile , two wheeler'],
    ['19', 'その他の製造業 / Manufacturing - others'],
    ['20', '電気・ガス・熱供給・水道業 / Electricity, gas, water supply'],
    ['21', '情報通信業 / Broadcasting, telecommunication etc.'],
    ['22', '運輸業 / Transportation'],
    ['23', '卸売業（商社） / Wholesale (trading company)'],
    ['24', '卸売業（販社） / Wholesale (sales company)'],
    ['25', '小売業 / Retail'],
    ['26', '金融業、保険業 / Finance and insurance'],
    ['27', '不動産業 / Real estate'],
    ['28', '物品賃貸業 / Product lease'],
    ['29', '宿泊業、飲食サービス業 / Hotel service, restaurant'],
    ['30', '教育、学習支援サービス業 / Education, learning support service'],
    ['31', '医療、福祉サービス業 / Medical, social welfare service'],
    ['32',
    '複合サービス業 / Compound service (postal, cooperative, association etc.)'],
    ['33', 'その他のサービス業 / Other service']
  ],

  columns=['biz_type_code', 'biz_description']
)

table_biz_types

,biz_type_code,biz_description
0,00,(blank)
1,01,"農業、林業、漁業 / Agriculture, forestry, fishery"
2,02,(blank)
3,03,建設業 / Construction industry
4,04,"食料品、飲料・たばこ・飼料製造業 / Manufacturing - Food, drink..."
5,05,繊維工業 / Textile industry
6,06,木材・木製品、パルプ・紙・紙加工品製造業 / Manufacturing - paper a...
7,07,化学工業 / Chemical industry
8,08,"石油製品・石炭製品製造業 / Manufacturing - coal products, ..."
9,09,窯業・土石製品製造業 / Pottery


## Non_numeric系の整理
- Non_numeric系も [Triad退治](https://ggcs.io/2020/08/11/google-colab-pdf-export-02#scr004) の副作用で生じたもので、biz_type に修正が必要ならば、同じ row の他の column もほぼ確実に修正が必要だと推測されるので、Non_numeric系の row については、
1. Non_numeric系（biz_type column のセルの先頭が***数字ではない***もの）の list を作り（Script 5）、
2. list に基づいて DFs を抽出し（Script 6）、
3. DFs を単一の DF にまとめた（Script 7）上で CSV に落とします（Script 8）。

- で、オリジナル PDF と付き合わせてまた手作業で修正 😭

In [ ]:
#Script 5

# Non_numeric系の list 化。
# list_biz_type から iセルを取り出し、最初の文字（i[0]）が数字じゃなかったら
# そいつは Non_numeric系だから、list に加える。
list_btyp_non_num = [i for i in list_biz_type if not i[0].isnumeric()]
list_btyp_non_num

In [ ]:
# Script 6

# biz_type column の内容がリストと一致する DFs を抽出、list 化
list_df_btyp_non_num = list(
    map(lambda x: df[df.biz_type == x], list_btyp_non_num))

# Script 7
# 抽出した DFs をひとつの DF にまとめる。
df_btyp_non_num = pd.concat(list_df_btyp_non_num)

# いろいろやって順序が乱れているので sort しておいた方が見やすい。
df_btyp_non_num.sort_values(by='id', inplace=True)

# Script 8
# 手作業用の CSV ファイルに落とす。
df_btyp_non_num.to_csv(
    os.path.join(DATA_PATH, 'btyp_non_num.csv'), index=True)
# 確認
df_btyp_non_num

,id,state_code,location,sc_num,com_in,com_jp,biz_type
346,346,HR,Bawal,7,"Caparo MI Steel Processing, Denso Ten Minda",伊藤忠丸紅鉄鋼,"wheeler, 10 鉄鋼業/ Steel, 18 輸送機械器具製造業/ Manufa..."
351,351,HR,Bawal,12,HANKYU HANSHIN EXPRESS INDIA,阪急阪神エクスプレス,"wheeler, 22 運輸業/ Transportation, 18 輸送機械器具製造業..."
365,365,HR,Bawal,26,RANE NSK STEERING SYSTEMS,日本精工,"wheeler, 12 金属製品製造業/ Manufacturing metal prod..."
367,367,HR,Bawal,28,Sanko Gosei Technology India,"三光合成, 三光合成","wheeler, 07 化学工業/ Chemical industry, 14 生産用機械..."
374,374,HR,Dharuhera,35,"Dharuhera Plant, SMI AMTEK Crankshaft",新日鐵住金,"wheeler, 12 金属製品製造業/ Manufacturing metal prod..."
2397,2397,MH,Mumbai,273,Lintec India,"リンテック, メタルワン","machinery, 24 卸売業(販社)/ Wholesale (sales comp..."
4201,4201,TN,Chennai,176,Kohyei Polymers India,弘栄貿易,"/ Wholesale (trading company), 11 非鉄金属製造業/ M..."
4250,4250,TN,Chennai,225,Mizuho Bank,みずほ銀行,"machinery, 26 金融業、保険業/ Finance and insurance,..."
4259,4259,TN,Chennai,234,MUFG Bank,"ミテッド, 三菱UFJ銀行, 村田製作所","/ Wholesale (trading company), 26 金融業、保険業/ F..."
4261,4261,TN,Chennai,236,Murugappa Organo Water Solutions,"村田製作所シンガポール, オルガノ",Manufacturing- IT products and electronic prod...


- 上記の btyp_non_num.csv を手作業で修正したものがこちら。

In [ ]:
# Script 9

# df_btyp_non_num を手作業で修正（correct）した DF.
df_btyp_non_num_corrected = pd.DataFrame(
  [
    [346, 'HR', 'Bawal', 7,
    'Caparo MI Steel Processing, Denso Ten Minda', '伊藤忠丸紅鉄鋼',
    '10 鉄鋼業/  Steel'],
    [351, 'HR', 'Bawal', 12, 'HANKYU HANSHIN EXPRESS INDIA',
    '阪急阪神エクスプレス', '22 運輸業/  Transportation'],
    [365, 'HR', 'Bawal', 26, 'RANE NSK STEERING SYSTEMS', '日本精工',
    '12 金属製品製造業/  Manufacturing metal products'],
    [367, 'HR', 'Bawal', 28, 'Sanko Gosei Technology India', '三光合成',
    '07 化学工業/  Chemical industry'],
    [374, 'HR', 'Dharuhera', 35, 'SMI AMTEK Crankshaft', '新日鐵住金',
    '12 金属製品製造業/  Manufacturing metal products'],
    [2397, 'MH', 'Mumbai', 273, 'Lintec India', 'リンテック',
    '24  卸売業(販社)/  Wholesale (sales company)'],
    [4201, 'TN', 'Chennai', 176, 'Kohyei Polymers India', '弘栄貿易',
    '11 非鉄金属製造業/  Manufacturing -  Non-metal products'],
    [4250, 'TN', 'Chennai', 225, 'Mizuho Bank', 'みずほ銀行',
    '26 金融業、保険業/  Finance and insurance'],
    [4259, 'TN', 'Chennai', 234, 'MUFG Bank', '三菱UFJ銀行',
    '26 金融業、保険業/  Finance and insurance'],
    [4261, 'TN', 'Chennai', 236, 'Murugappa Organo Water Solutions',
    'オルガノ', '01  農業、林業、漁業/  Agriculture, forestry, fishery'],
    [4264, 'TN', 'Chennai', 239, 'Netmagic Solutions  Chennai Office',
    'NTTコミュニケーションズ',
    '21 情報通信業/  Broadcasting, telecommunication etc.'],
    [4282, 'TN', 'Chennai', 257, 'NISSIN ABC LOGISTICS', '日新',
    '22 運輸業/  Transportation']
  ],

  columns=['id', 'state_code', 'location', 'sc_num', 
          'com_in', 'com_jp', 'biz_type']
)

# あとあとの都合があるので、index を key の値に一致させておきます。
df_btyp_non_num_corrected.index = df_btyp_non_num_corrected.id

# 確認
df_btyp_non_num_corrected

,id,state_code,location,sc_num,com_in,com_jp,biz_type
id,,,,,,,
346,346,HR,Bawal,7,"Caparo MI Steel Processing, Denso Ten Minda",伊藤忠丸紅鉄鋼,10 鉄鋼業/ Steel
351,351,HR,Bawal,12,HANKYU HANSHIN EXPRESS INDIA,阪急阪神エクスプレス,22 運輸業/ Transportation
365,365,HR,Bawal,26,RANE NSK STEERING SYSTEMS,日本精工,12 金属製品製造業/ Manufacturing metal products
367,367,HR,Bawal,28,Sanko Gosei Technology India,三光合成,07 化学工業/ Chemical industry
374,374,HR,Dharuhera,35,SMI AMTEK Crankshaft,新日鐵住金,12 金属製品製造業/ Manufacturing metal products
2397,2397,MH,Mumbai,273,Lintec India,リンテック,24 卸売業(販社)/ Wholesale (sales company)
4201,4201,TN,Chennai,176,Kohyei Polymers India,弘栄貿易,11 非鉄金属製造業/ Manufacturing - Non-metal products
4250,4250,TN,Chennai,225,Mizuho Bank,みずほ銀行,26 金融業、保険業/ Finance and insurance
4259,4259,TN,Chennai,234,MUFG Bank,三菱UFJ銀行,26 金融業、保険業/ Finance and insurance


# com_jp column の Cleansing
- 前々回（死闘篇）で[セル内改行対策](https://ggcs.io/2020/08/11/google-colab-pdf-export-02#strange_r)を行ったため、com_jp column の中身も少し乱れています。
- 目視で確認したところ、複数の親会社（例えば、Ａ社とＢ社）を持つ場合に、'Ａ社&lt;br&gt;Ｂ社' となっているパターンと、'Ａ社及び&lt;br&gt;Ｂ社' となっているパターンの 2 パターンがあったので、これらは差し当たりすべて comma 区切りに変換しておきます。

In [ ]:
# Script 10

# Regex で、この順序で置換。
df.com_jp.replace(r'<br>', r', ', inplace=True, regex=True)
df.com_jp.replace(r'及び', r'', inplace=True, regex=True)
df.head(10)

# ☕️ ちょっと休憩

# CREATE a Master table
- 以上の作業結果をひとつの DF (master_table) にまとめます。

In [ ]:
# Script 11

# 新たに table_master (DataFrame) を作って気分一新。
table_master = df.copy()

## Non_numeric系修正結果反映
- 上の ***Non_numeric系の整理*** で、手作業で修正しておいた df_btyp_non_num_corrected を使って master_table を UPDATE します。

In [ ]:
# Script 12

# 修正済みの df_btyp_non_num_corrected で df_master を　UPDATE.
table_master.update(df_btyp_non_num_corrected, join='left')

# 修正結果を確認 -> 概ね OK. ただし、なぜか id と sc_num の dtype が float に
# なっているので、CSV 保存のタイミングで int に戻しておきます。
pd.concat(
  list(
    map(lambda x: table_master[table_master.id == x], df_btyp_non_num.id)
  )
)

## biz_type column の正規化
- Database に流し込むときに、biz_type は上で作った別 table (table_biz_types) に分けておいた方が便利なので、新たに 'biz_type_code column' を作って biz_type の先頭 2 桁の数字を入れておきます。

In [ ]:
# Script 13

# 業種を番号で管理したいので、
# 新しく biz_type_code column を作って biz_type の頭の数字 2 桁を入れておきます。
table_master['biz_type_code'] = table_master.biz_type.str[0:2]

table_master.head()

,id,state_code,location,sc_num,com_in,com_jp,biz_type,biz_type_code
0,0.0,DL,Delhi,1.0,"""K"" Line (Inia)",川崎汽船,22 運輸業/ Transportation,22
1,1.0,DL,Delhi,2.0,Agilion Energy,"三菱商事, AES","20 電気・ガス・熱供給・水道業/ Electricity, gas, water supply",20
2,2.0,DL,Delhi,3.0,Aica Laminates India,アイカ工業,07 化学工業/ Chemical industry,07
3,3.0,DL,Delhi,4.0,Anchor Electricals,パナソニック,16 電気機械器具製造業/ Manufacturing-electronic machinery,16
4,4.0,DL,Delhi,5.0,Asahi Modi Materials,"旭有機材, モディーラバー",07 化学工業/ Chemical industry,07


In [ ]:
# Script 14

# biz_type_code を KEY にして LEFT JOIN する。
table_master = pd.merge(table_master, table_biz_types, how='left', on='biz_type_code')
table_master.head()

,id,state_code,location,sc_num,com_in,com_jp,biz_type,biz_type_code,biz_description
0,0.0,DL,Delhi,1.0,"""K"" Line (Inia)",川崎汽船,22 運輸業/ Transportation,22,運輸業 / Transportation
1,1.0,DL,Delhi,2.0,Agilion Energy,"三菱商事, AES","20 電気・ガス・熱供給・水道業/ Electricity, gas, water supply",20,"電気・ガス・熱供給・水道業 / Electricity, gas, water supply"
2,2.0,DL,Delhi,3.0,Aica Laminates India,アイカ工業,07 化学工業/ Chemical industry,07,化学工業 / Chemical industry
3,3.0,DL,Delhi,4.0,Anchor Electricals,パナソニック,16 電気機械器具製造業/ Manufacturing-electronic machinery,16,電気機械器具製造業 / Manufacturing - electronic machinery
4,4.0,DL,Delhi,5.0,Asahi Modi Materials,"旭有機材, モディーラバー",07 化学工業/ Chemical industry,07,化学工業 / Chemical industry


- 目視確認したところ、biz_type_code と biz_description の columns は正常に JOIN されたようなので、biz_type column（重複した情報を含んでいる） は drop しておきます。

In [ ]:
# Script 15

table_master.drop(columns=['biz_type'], inplace=True)
table_master.head()

# CSV ファイルに保存
- table_master と table_biz_types を CSV ファイルに保存します。
- Database に納める時は、
1. table_master の biz_description 以外の columns と、
2. table_biz_types の 2 つの table を CREATE して、
3. biz_type_code を FOREIGN KEY に指定すると良いと思います。

In [ ]:
# Script 16

# key, sc_num の dtype（の見た目）を int にしておく。
table_master[['id', 'sc_num']] =\
 table_master[['id', 'sc_num']].astype('int')

# biz_type_code dtype（の見た目）を string にしておく。
'''できあがった CSV を editor で開くとわかりますが、1 桁の数字は、01, ... 09
という感じに期待通り padding されています。しかし、CSV を開くアプリによっては、
せっかくのゼロを勝手に削除してしまうものもあるので注意が必要です。'''
table_master[['biz_type_code']] =\
table_master[['biz_type_code']].astype('str')

table_biz_types[['biz_type_code']] =\
 table_biz_types[['biz_type_code']].astype('str')

In [ ]:
table_master.to_csv(os.path.join(DATA_PATH, 'table_master.csv'), index=False)
table_biz_types.to_csv(os.path.join(DATA_PATH, 'table_biz_types.csv'), index=False)
print('DONE! お疲れ様でした 🍻')